# Install Libraries

In [1]:
# !pip install tensorflow pandas numpy sklearn
# !pip install pandas scikit-learn
# !pip install lime shap
# !pip install scikeras

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 kB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 532.9/532.9 kB 23.5 MB/s eta 0:00:00
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283834 sha256=51034975a1a66790888f195087445198423153ee5de1ef96b17767767f45412e
  Stored in directory: /root/.cache/pip/wheels/fd/a2/af/9ac0a1a85a27f314a06b39e1f492bee1547d52549a4606ed89
Successfully built lime


# Import Libraries

In [2]:
import pandas as pd
import numpy as np
import pandas as pd
from datetime import timedelta
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.metrics import AUC
from tensorflow.keras.metrics import Precision, TruePositives, TrueNegatives, FalsePositives, FalseNegatives, AUC
from sklearn.metrics import accuracy_score, recall_score, roc_auc_score
from keras.layers import SimpleRNN, Dense
import random
import tensorflow as tf
from google.colab import drive

# Read Preprocessed Data

In [3]:
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
file_path_AU_preprocessed = '/content/drive/MyDrive/CKD/GitHub/data/preprocessed/AU/AU_preprocessed_df.csv'
file_path_JP_preprocessed = '/content/drive/MyDrive/CKD/GitHub/data/preprocessed/JP/JP_preprocessed_df.csv'

# Train and Evaluate Models

### TUNED HYPERAMETER MODEL - AU Dataset

In [18]:
# Read Data
AU_df = pd.read_csv(file_path_AU_preprocessed)
AU_df['COLLECTED'] = pd.to_datetime(AU_df['COLLECTED'])
AU_df = AU_df.reset_index(drop=True)

# Normalize the numeric columns
scaler = MinMaxScaler()
AU_df[['AGE', 'eGFR']] = scaler.fit_transform(AU_df[['AGE', 'eGFR']])

# Filter UIDs with sequences >= 3
grouped = AU_df.groupby(['ID', 'Kidney_Failure']).size().unstack(fill_value=0)
grouped = grouped[(grouped[1] >= 0) | (grouped[0] >= 0)] #3

# List of columns to drop
cols_to_drop = ['ID', 'Kidney_Failure', 'COLLECTED']  # add other non-numeric columns if present

X = pad_sequences(AU_df.groupby('ID').apply(lambda group: group.drop(cols_to_drop, axis=1).to_numpy()).tolist(), maxlen=50, dtype='float32', padding='post')
y = AU_df.groupby('ID')['Kidney_Failure'].first().to_numpy()

# Initialise class weights
class_counts = np.bincount(y)
total_samples = np.sum(class_counts)
class_weights = {0: total_samples / (class_counts[0] + 1), 1: total_samples / (class_counts[1] + 1)}

In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score, f1_score
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense
from keras.optimizers import Adam
from scikeras.wrappers import KerasClassifier, KerasRegressor

# Set random seeds
seed_value = 3
np.random.seed(seed_value)

# Adjust this threshold as needed
threshold = 0.5

# Create a dataframe to store the results
df_results = pd.DataFrame(columns=['TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives',
                                   'TPR', 'TNR', 'FPR', 'FNR',
                                   'Accuracy', 'Precision', 'Recall', 'f1_score', 'Specificity', 'ROC-AUC'])

# Function to create the model, required for KerasClassifier
def create_model(lr=0.001):
    # Create model
    model = Sequential()
    model.add(SimpleRNN(units=3, input_shape=(50, 3), return_sequences=False))
    model.add(Dense(1, activation='sigmoid'))
    optimizer = Adam(learning_rate=lr)
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=Precision())
    return model

# Create the KerasClassifier
model = KerasClassifier(build_fn=create_model, epochs=20, batch_size=128, lr=0.001, verbose=1)

# Define the grid search parameters
param_grid = {
    #'epochs': [10, 50],
    #'batch_size': [32, 128],
    'model__lr': [0.001, 0.01, 0.1] # Prefix with 'model__' to denote it's a parameter for the model-building function
}

#grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid = GridSearchCV(estimator=model, param_grid=param_grid, scoring='precision', n_jobs=-1, cv=3)

grid_result = grid.fit(X, y)

# Output the best parameters and score
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

/usr/local/lib/python3.10/dist-packages/scikeras/wrappers.py:915: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)


Epoch 1/20
79/79 [==============================] - 5s 28ms/step - loss: 0.5962 - precision: 0.0646
Epoch 2/20
79/79 [==============================] - 2s 23ms/step - loss: 0.4852 - precision: 0.0000e+00
Epoch 3/20
79/79 [==============================] - 2s 21ms/step - loss: 0.3939 - precision: 0.0000e+00
Epoch 4/20
79/79 [==============================] - 1s 16ms/step - loss: 0.3191 - precision: 0.0000e+00
Epoch 5/20
79/79 [==============================] - 1s 9ms/step - loss: 0.2619 - precision: 0.0000e+00
Epoch 6/20
79/79 [==============================] - 1s 9ms/step - loss: 0.2214 - precision: 0.0000e+00
Epoch 7/20
79/79 [==============================] - 1s 9ms/step - loss: 0.1948 - precision: 0.0000e+00
Epoch 8/20
79/79 [==============================] - 1s 9ms/step - loss: 0.1783 - precision: 0.0000e+00
Epoch 9/20
79/79 [==============================] - 1s 9ms/step - loss: 0.1682 - precision: 0.0000e+00
Epoch 10/20
79/79 [==============================] - 1s 9ms/step - loss: 

In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, precision_score, recall_score, accuracy_score, f1_score

# Set random seeds
seed_value = 0
random.seed(seed_value)
np.random.seed(seed_value)

# Adjust this threshold as needed
threshold = 0.5

# Create a dataframe to store the results
df_results = pd.DataFrame(columns=['TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives',
                                   'TPR', 'TNR', 'FPR', 'FNR',
                                   'Accuracy', 'Precision', 'Recall', 'f1_score', 'Specificity', 'ROC-AUC'])

# Use Stratified KFold for cross-validation
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=seed_value)

for train_idx, test_idx in skf.split(X, y):
    X_train, X_test = X[train_idx], X[test_idx]
    y_train, y_test = y[train_idx], y[test_idx]

    # Define the model
    model = Sequential()
    model.add(SimpleRNN(units=3, input_shape=(50, 3), return_sequences=False)) #50, 4 !CHECK
    model.add(Dense(1, activation = 'sigmoid')) # This is just a linear combination

    # Compile the model
    precision = Precision(thresholds=threshold)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[precision])

    # Fit the model
    model.fit(X_train, y_train, epochs=50, batch_size=128, class_weight=class_weights)
    print(class_weights)

    # Evaluate the model on the test set
    loss = model.evaluate(X_test, y_test)

    # Get the predictions
    predictions = model.predict(X_test)
    predicted_labels = (predictions >= threshold).astype(int).flatten()

    # Compute the statistics
    true_positives = np.sum((predicted_labels == 1) & (y_test == 1))
    true_negatives = np.sum((predicted_labels == 0) & (y_test == 0))
    false_positives = np.sum((predicted_labels == 1) & (y_test == 0))
    false_negatives = np.sum((predicted_labels == 0) & (y_test == 1))

    tpr = true_positives / (true_positives + false_negatives)
    tnr = true_negatives / (true_negatives + false_positives)
    fpr = false_positives / (false_positives + true_negatives)
    fnr = false_negatives / (false_negatives + true_positives)
    accuracy = (true_positives + true_negatives) / len(y_test)
    precision = precision_score(y_test, predicted_labels)
    recall = recall_score(y_test, predicted_labels)
    f1 = 2 * precision * recall / (precision + recall)
    specificity = true_negatives / (true_negatives + false_positives)
    roc_auc = roc_auc_score(y_test, predictions)

    print(accuracy, precision, recall, f1, specificity, roc_auc)

    # Add the statistics to the dataframe
    df_results.loc[len(df_results)] = [true_positives, true_negatives, false_positives, false_negatives, tpr, tnr, fpr, fnr, accuracy, precision, recall, f1, specificity, roc_auc]

# Print the average of each statistic over the 5 folds
mean_results = df_results.mean()
std_results = df_results.std()

# Create a new DataFrame to display mean and standard deviation
results_summary = pd.DataFrame(index=mean_results.index)
results_summary['Mean'] = mean_results
results_summary['Standard Deviation'] = std_results

print(results_summary)

Epoch 1/50
63/63 [==============================] - 4s 22ms/step - loss: 1.3494 - precision_7: 0.0504
Epoch 2/50
63/63 [==============================] - 1s 10ms/step - loss: 1.3355 - precision_7: 0.0552
Epoch 3/50
63/63 [==============================] - 1s 9ms/step - loss: 1.3286 - precision_7: 0.0550
Epoch 4/50
63/63 [==============================] - 1s 9ms/step - loss: 1.3165 - precision_7: 0.0598
Epoch 5/50
63/63 [==============================] - 1s 10ms/step - loss: 1.3086 - precision_7: 0.0581
Epoch 6/50
63/63 [==============================] - 1s 9ms/step - loss: 1.2934 - precision_7: 0.0683
Epoch 7/50
63/63 [==============================] - 1s 14ms/step - loss: 1.2815 - precision_7: 0.0634
Epoch 8/50
63/63 [==============================] - 1s 23ms/step - loss: 1.2688 - precision_7: 0.0680
Epoch 9/50
63/63 [==============================] - 2s 35ms/step - loss: 1.2569 - precision_7: 0.0741
Epoch 10/50
63/63 [==============================] - 1s 18ms/step - loss: 1.2504 - pr

In [20]:
# Train the final model on the full Australian dataset

import numpy as np
from keras.models import Sequential
from keras.layers import SimpleRNN, Dense
from keras.metrics import Precision

# Set random seeds for reproducibility
seed_value = 0
random.seed(seed_value)
np.random.seed(seed_value)

# Adjust this threshold as needed
threshold = 0.5

# Define the RNN model for the entire dataset
model = Sequential()
model.add(SimpleRNN(units=3, input_shape=(50, 3), return_sequences=False))
model.add(Dense(1, activation='sigmoid')) # This is a binary classification output

# Compile the model
precision = Precision(thresholds=threshold)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[precision])

# Fit the model on the entire dataset
model.fit(X, y, epochs=50, batch_size=128, class_weight=class_weights)

Epoch 1/50
79/79 [==============================] - 6s 15ms/step - loss: 1.3597 - precision_12: 0.0519
Epoch 2/50
79/79 [==============================] - 1s 10ms/step - loss: 1.3457 - precision_12: 0.0517
Epoch 3/50
79/79 [==============================] - 1s 10ms/step - loss: 1.3314 - precision_12: 0.0554
Epoch 4/50
79/79 [==============================] - 1s 10ms/step - loss: 1.3216 - precision_12: 0.0647
Epoch 5/50
79/79 [==============================] - 1s 13ms/step - loss: 1.3005 - precision_12: 0.0636
Epoch 6/50
79/79 [==============================] - 2s 21ms/step - loss: 1.2751 - precision_12: 0.0790
Epoch 7/50
79/79 [==============================] - 1s 10ms/step - loss: 1.2467 - precision_12: 0.1010
Epoch 8/50
79/79 [==============================] - 1s 11ms/step - loss: 1.2253 - precision_12: 0.1094
Epoch 9/50
79/79 [==============================] - 1s 10ms/step - loss: 1.2170 - precision_12: 0.1135
Epoch 10/50
79/79 [==============================] - 1s 10ms/step - loss:

### HELD-OUT TEST DATA - Tuned Model (Without Fine-Tuning)

In [26]:
from pandas.tseries.offsets import DateOffset
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

JP_df = pd.read_csv(file_path_JP_preprocessed)
JP_df = JP_df.reset_index(drop=True)

# Function to convert Time to months
def convert_to_months(time_str):
    if time_str == 'eGFR(last visit)':
        return 42
    else:
        return int(time_str.split('(')[-1].split('M')[0])

# Apply the function to the 'Time' column to create a new 'Months' column
JP_df['Months'] = JP_df['Time'].apply(convert_to_months)

# Assume a starting date for all IDs
start_date = pd.Timestamp('2020-01-01')

# Create a 'COLLECTED' column based on the 'Months' column
JP_df['COLLECTED'] = JP_df.apply(lambda row: start_date + DateOffset(months=row['Months']), axis=1)

#JP_df.drop(['Time', 'Months'], inplace = True)

# Normalize the numeric columns
scaler = MinMaxScaler()
JP_df[['AGE', 'eGFR']] = scaler.fit_transform(JP_df[['AGE', 'eGFR']])

# Filter UIDs with sequences >= 3
grouped = JP_df.groupby(['ID', 'Kidney_Failure']).size().unstack(fill_value=0)
grouped = grouped[(grouped[1] >= 0) | (grouped[0] >= 0)] #3

# List of columns to drop
cols_to_drop = ['ID', 'Kidney_Failure', 'COLLECTED', 'Time', 'Months']  # add other non-numeric columns if present

X = pad_sequences(JP_df.groupby('ID').apply(lambda group: group.drop(cols_to_drop, axis=1).to_numpy()).tolist(), maxlen=50, dtype='float32', padding='post')
y = JP_df.groupby('ID')['Kidney_Failure'].first().to_numpy()

# Initialise class weights
class_counts = np.bincount(y)
total_samples = np.sum(class_counts)
class_weights = {0: total_samples / (class_counts[0] + 1), 1: total_samples / (class_counts[1] + 1)}

In [23]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import classification_report, roc_auc_score, precision_score, recall_score, accuracy_score
import random

# Set random seeds
seed_value = 0
random.seed(seed_value)
np.random.seed(seed_value)

# Adjust this threshold as needed
threshold = 0.5

# Evaluate on the held-out test set
#clf = best_clf

# Evaluate the model on the test set
# loss = model.evaluate(X, y)

y_pred = model.predict(X)

predicted_labels = (y_pred >= threshold).astype(int).flatten()

# Compute the statistics
true_positives = np.sum((predicted_labels == 1) & (y == 1))
true_negatives = np.sum((predicted_labels == 0) & (y == 0))
false_positives = np.sum((predicted_labels == 1) & (y == 0))
false_negatives = np.sum((predicted_labels == 0) & (y == 1))

tpr = true_positives / (true_positives + false_negatives)
tnr = true_negatives / (true_negatives + false_positives)
fpr = false_positives / (false_positives + true_negatives)
fnr = false_negatives / (false_negatives + true_positives)
accuracy = (true_positives + true_negatives) / len(y)
precision = precision_score(y, predicted_labels)
recall = recall_score(y, predicted_labels)
f1 = 2 * precision * recall / (precision + recall)
specificity = true_negatives / (true_negatives + false_positives)
roc_auc = roc_auc_score(y, predicted_labels)
#roc_auc = roc_auc_score(y, clf.predict_proba(X)[:, 1])

print(accuracy, precision, recall, f1, specificity, roc_auc)

print("TPR: {:.6g}".format(tpr), "TNR: {:.6g}".format(tnr), "FPR: {:.6g}".format(fpr), "FNR: {:.6g}".format(fnr),
      '\n' "Accuracy: {:.6g}".format(accuracy), "Precision: {:.6g}".format(precision), "Recall: {:.6g}".format(recall), "F1 Score: {:.6g}".format(f1_score), "Specificity: {:.6g}".format(specificity), "ROC-AUC: {:.6g}".format(roc_auc))

19/19 [==============================] - 0s 4ms/step
0.6700167504187605 0.45098039215686275 0.9938271604938271 0.6204238921001928 0.5494252873563218 0.7716262239250745
TPR: 0.9938271604938271 TNR: 0.5494252873563218 FPR: 0.45057471264367815 FNR: 0.006172839506172839 
Accuracy 0.6700167504187605 Precision: 0.45098039215686275 Recall: 0.9938271604938271 F1 Score: 0.6204238921001928 Specificity: 0.5494252873563218 ROC-AUC: 0.7716262239250745


### HELD-OUT TEST DATA - Tuned Model (with Fine Tuning)

In [24]:
from pandas.tseries.offsets import DateOffset
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

JP_df = pd.read_csv(file_path_JP_preprocessed)
JP_df = JP_df.reset_index(drop=True)

# Function to convert Time to months
def convert_to_months(time_str):
    if time_str == 'eGFR(last visit)':
        return 42
    else:
        return int(time_str.split('(')[-1].split('M')[0])

# Apply the function to the 'Time' column to create a new 'Months' column
JP_df['Months'] = JP_df['Time'].apply(convert_to_months)

# Assume a starting date for all IDs
start_date = pd.Timestamp('2020-01-01')

# Create a 'COLLECTED' column based on the 'Months' column
JP_df['COLLECTED'] = JP_df.apply(lambda row: start_date + DateOffset(months=row['Months']), axis=1)

#JP_df.drop(['Time', 'Months'], inplace = True)

# Normalize the numeric columns
scaler = MinMaxScaler()
JP_df[['AGE', 'eGFR']] = scaler.fit_transform(JP_df[['AGE', 'eGFR']])

# Filter UIDs with sequences >= 3
grouped = JP_df.groupby(['ID', 'Kidney_Failure']).size().unstack(fill_value=0)
grouped = grouped[(grouped[1] >= 0) | (grouped[0] >= 0)] #3

# List of columns to drop
cols_to_drop = ['ID', 'Kidney_Failure', 'COLLECTED', 'Time', 'Months']  # add other non-numeric columns if present

X = pad_sequences(JP_df.groupby('ID').apply(lambda group: group.drop(cols_to_drop, axis=1).to_numpy()).tolist(), maxlen=50, dtype='float32', padding='post')
y = JP_df.groupby('ID')['Kidney_Failure'].first().to_numpy()

# Initialise class weights
class_counts = np.bincount(y)
total_samples = np.sum(class_counts)
class_weights = {0: total_samples / (class_counts[0] + 1), 1: total_samples / (class_counts[1] + 1)}


In [25]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import classification_report, roc_auc_score, precision_score, recall_score, accuracy_score
import random
from sklearn import metrics
import matplotlib.pyplot as plt

# Set random seeds
seed_value = 0
random.seed(seed_value)
np.random.seed(seed_value)

# Adjust this threshold as needed
threshold = 0.5

# Train the model on 15% of the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.85, random_state=seed_value, stratify=y)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

predicted_labels = (y_pred >= threshold).astype(int).flatten()

# Compute the statistics
true_positives = np.sum((predicted_labels == 1) & (y_test == 1))
true_negatives = np.sum((predicted_labels == 0) & (y_test == 0))
false_positives = np.sum((predicted_labels == 1) & (y_test == 0))
false_negatives = np.sum((predicted_labels == 0) & (y_test == 1))

tpr = true_positives / (true_positives + false_negatives)
tnr = true_negatives / (true_negatives + false_positives)
fpr = false_positives / (false_positives + true_negatives)
fnr = false_negatives / (false_negatives + true_positives)
accuracy = (true_positives + true_negatives) / len(y_test)
precision = precision_score(y_test, predicted_labels)
recall = recall_score(y_test, predicted_labels)
f1 = 2 * precision * recall / (precision + recall)
specificity = true_negatives / (true_negatives + false_positives)
roc_auc = roc_auc_score(y_test, predicted_labels)

print(accuracy, precision, recall, f1, specificity, roc_auc)

print("TPR: {:.6g}".format(tpr), "TNR: {:.6g}".format(tnr), "FPR: {:.6g}".format(fpr), "FNR: {:.6g}".format(fnr),
      '\n' "Accuracy: {:.6g}".format(accuracy), "Precision: {:.6g}".format(precision), "Recall: {:.6g}".format(recall), "F1 Score: {:.6g}".format(f1_score), "Specificity: {:.6g}".format(specificity), "ROC-AUC: {:.6g}".format(roc_auc))

16/16 [==============================] - 0s 4ms/step
0.6948818897637795 0.47079037800687284 0.9927536231884058 0.6386946386946386 0.5837837837837838 0.7882687034860948
TPR: 0.9927536231884058 TNR: 0.5837837837837838 FPR: 0.41621621621621624 FNR: 0.007246376811594203 
Accuracy 0.6948818897637795 Precision: 0.47079037800687284 Recall: 0.9927536231884058 F1 Score: 0.6386946386946386 Specificity: 0.5837837837837838 ROC-AUC: 0.7882687034860948
